<a href="https://colab.research.google.com/github/ssd909/first_step_in-ml/blob/master/tr_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

url = 'https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv'
df = pd.read_csv(url)


print(df)
print(df.shape)

In [ ]:
df.tail()

In [ ]:
print(df.shape)

In [ ]:
print(df.dtypes)

In [ ]:
print(df.columns)

In [ ]:
numeric_columns=df.select_dtypes(include=['int64','float64']).columns
object_columns=df.select_dtypes(include=['object']).columns
print(numeric_columns)
print(object_columns)

In [ ]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

In [ ]:
print(df.isnull().sum() )

In [ ]:
df["TotalCharges"].fillna(df["TotalCharges"].median(), inplace=True)
df["TotalCharges"].isna().sum()

In [ ]:
print(df[numeric_columns].describe())

In [ ]:
from scipy.stats import skew, kurtosis
for col in numeric_columns:
    print("\n")
    print(col)
    print("Skewness:", skew(df[col]))
    print("Kurtosis:", kurtosis(df[col]))


In [ ]:
def Eda_summary(data):
    print("INFO")
    print(df.info())

    print("\nDESCRIBE")
    print(df.describe())

    print("\nMISSING VALUES")
    print(df.isnull().sum())

    print("\nCORRELATION")
    print(df.corr(numeric_only=True))

Eda_summary(df)

In [ ]:
df[['SeniorCitizen', 'tenure', 'MonthlyCharges']].cov()


In [ ]:
df[['SeniorCitizen', 'tenure', 'MonthlyCharges']].corr()

In [ ]:
df.info()

In [ ]:
df.drop("customerID",axis=1,inplace=True)

In [ ]:
print(df.info())



customerID ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
--------------------------------------------------
gender ['Female' 'Male']
--------------------------------------------------
Partner ['Yes' 'No']
--------------------------------------------------
Dependents ['No' 'Yes']
--------------------------------------------------
PhoneService ['No' 'Yes']
--------------------------------------------------
MultipleLines ['No phone service' 'No' 'Yes']
--------------------------------------------------
InternetService ['DSL' 'Fiber optic' 'No']
--------------------------------------------------
OnlineSecurity ['No' 'Yes' 'No internet service']
--------------------------------------------------
OnlineBackup ['Yes' 'No' 'No internet service']
--------------------------------------------------
DeviceProtection ['No' 'Yes' 'No internet service']
--------------------------------------------------
TechSupport ['No' 'Yes' 'No internet service']
--------------------------------------------------
StreamingTV ['No' 'Yes' 'No internet service']
--------------------------------------------------
StreamingMovies ['No' 'Yes' 'No internet service']
--------------------------------------------------
Contract ['Month-to-month' 'One year' 'Two year']
--------------------------------------------------
PaperlessBilling ['Yes' 'No']
--------------------------------------------------
PaymentMethod ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
--------------------------------------------------
Churn ['No' 'Yes']
--------------------------------------------------

---



In [ ]:
df['MultipleLines'].replace('No phone service','No',inplace=True)
df['OnlineSecurity'].replace('No internet service','No',inplace=True)
df['OnlineBackup'].replace('No internet service','No',inplace=True)
df['DeviceProtection'].replace('No internet service','No',inplace=True)
df['TechSupport'].replace('No internet service','No',inplace=True)
df['StreamingTV'].replace('No internet service','No',inplace=True)
df['StreamingMovies'].replace('No internet service','No',inplace=True)

In [ ]:
df.info()


In [ ]:
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']
for col in yes_no_columns:
    df[col].replace({'Yes': 1,'No': 0},inplace=True)

df.head(2)

In [ ]:
df['gender'].replace({'Female': 1,'Male': 0},inplace=True)
df.head(3)

In [ ]:
df_encoded = pd.get_dummies(df,drop_first=True)
df_encoded


In [ ]:
df_encoded = df_encoded.astype(int)
df_encoded


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [ ]:
x=df_encoded.drop('Churn',axis=1)
y=df_encoded['Churn']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


In [ ]:
print(y_train.value_counts())

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
x_train_sm, y_train_sm = smote.fit_resample(x_train, y_train)
print(y_train_sm.value_counts())

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_sm = scaler.fit_transform(x_train_sm)
x_test = scaler.transform(x_test)


In [ ]:
tree_model=RandomForestClassifier(random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
param_grid={
    'n_estimators':[50,1000,300,500],
    'criterion':['gini','entropy'],
    'max_depth':[2,3],
    'bootstrap':[True,False],
    'min_samples_split':[2,3],

}
random_ssearch=RandomizedSearchCV(
   estimator= tree_model,
   param_distributions= param_grid,
   scoring='recall_weighted',
   cv=10,
   n_jobs=-1,

)

In [ ]:
random_ssearch.fit(x_train_sm,y_train_sm)


In [ ]:
print(random_ssearch.best_params_)
print(random_ssearch.best_score_)

In [ ]:
train_model=RandomForestClassifier(
     random_state=42,
    n_estimators=random_ssearch.best_params_['n_estimators'],
    criterion=random_ssearch.best_params_['criterion'],
    max_depth=random_ssearch.best_params_['max_depth'],
    min_samples_split=random_ssearch.best_params_['min_samples_split'],
   bootstrap=random_ssearch.best_params_['bootstrap'],
)


In [ ]:
train_model.fit(x_train_sm,y_train_sm)

In [ ]:
y_pred=train_model.predict(x_test)
train_accuracy=train_model.score(x_train_sm,y_train_sm)
test_accuracy=train_model.score(x_test,y_test)
print(train_accuracy)
print(test_accuracy)

try xgboos


In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(random_state=42)




In [ ]:
xg_hip={

    'n_estimators': [100, 200,400],
    'learning_rate': [ 0.05, 0.1],


    'max_depth': [2,3],
    'min_child_weight': [ 5, 8],
    'gamma': [ 0.4,2,4],


    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],


    'reg_alpha': [0, 0.01, 0.1, 3], # L1
    'reg_lambda': [1, 1.5, 2, 5]     # L2

}
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=xg_hip,
    n_iter=10,
    scoring='accuracy',
    cv=10,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

In [ ]:
random_search.fit(x_train_sm, y_train_sm)

In [ ]:
print(random_search.best_params_)
print(random_search.best_score_)

In [ ]:
train_xgb_model=XGBClassifier(
  random_state=42,
  n_estimators=random_search.best_params_['n_estimators'],
  learning_rate=random_search.best_params_['learning_rate'],
  max_depth=random_search.best_params_['max_depth'],
  min_child_weight=random_search.best_params_['min_child_weight'],
  gamma=random_search.best_params_['gamma'],
  subsample=random_search.best_params_['subsample'],
  colsample_bytree=random_search.best_params_['colsample_bytree'],
  reg_alpha=random_search.best_params_['reg_alpha'],
  reg_lambda=random_search.best_params_['reg_lambda'],


)


In [ ]:
train_xgb_model.fit(x_train_sm,y_train_sm)

In [ ]:
y_xg_pred=train_xgb_model.predict(x_test)
train_xg_accuracy=train_xgb_model.score(x_train_sm,y_train_sm)
test_xg_accuracy=train_xgb_model.score(x_test,y_test)
print(train_xg_accuracy)
print(test_xg_accuracy)

In [ ]:
!pip install catboost
from catboost import CatBoostClassifier

In [ ]:
cat_boost_model = CatBoostClassifier(
    random_state=42,
    iterations=2000,
    learning_rate=0.02,
    depth=3,
    l2_leaf_reg=3,
    random_strength=2,
    verbose=100,
    early_stopping_rounds=500


                                     )

In [ ]:
cat_boost_model.fit(x_train_sm, y_train_sm)

In [ ]:
y_pre_cat=cat_boost_model.predict(x_test)

In [ ]:
test_score=cat_boost_model.score(x_test,y_test)
train_score=cat_boost_model.score(x_train_sm,y_train_sm)
print(train_score)
print(test_score)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score

In [ ]:
print(accuracy_score(y_test,y_pre_cat))


In [ ]:
print(confusion_matrix(y_test,y_pre_cat))

In [ ]:
print(classification_report(y_test,y_pre_cat))